In [1]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
#using JWAS:misc               # utility functions
using JWAS.Datasets
using Distributions       
using Plots                   # package for plotting 
using CSV
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

In [2]:
phenotypes = CSV.read("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/production_1_7_NurPenBatch__weighted_Index_Nur+FinCL_Group_MV.dat",
types=Dict(:ID => String), delim = ' ',header=true, missingstrings=["NA"])
first(phenotypes,10)

,ID,Cycle,BatchNum,Batch,Company,SowID,NurEnrich,QNurPenBatch,NurPenBatch
,String,Int64,Int64,String,String,String,Bool,String,String?
1,0001,1,1,1B,A,1B_1715,false,1B_D04,1B_D04
2,0002,1,1,1B,A,1B_1642,false,1B_D04,1B_D04
3,0004,1,1,1B,A,1B_2150,false,1B_D04,1B_D04
4,0005,1,1,1B,A,1B_1980,false,1B_D04,1B_D04
5,0006,1,1,1B,A,1B_1902,false,1B_D04,1B_D04
6,0007,1,1,1B,A,1B_2150,false,1B_D04,1B_D04
7,0009,1,1,1B,A,1B_1902,false,1B_D02,1B_D02
8,0010,1,1,1B,A,1B_1885,false,1B_D02,1B_D02
9,0011,1,1,1B,A,1B_2097,false,1B_D02,1B_D02


In [17]:
phenotypes= dropmissing(phenotypes, :NurPenBatch)
phenotypes= dropmissing(phenotypes, :wt)

,ID,Cycle,BatchNum,Batch,Company,SowID,NurEnrich,QNurPenBatch,NurPenBatch
,SubStri…,Int64,Int64,String,String,String,Bool,String,String


In [4]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end
@time M = readMatBin("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/ALGP2_Cycle_1_7_Geno_Complete_Impute_JWAS.bin.gen.bin")

 36.504724 seconds (37.79 k allocations: 10.393 GiB, 0.11% gc time)


3205×435172 Array{Float64,2}:
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  2.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 1.0  1.0  0.0  1.0  2.0  0.0  0.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [5]:
rowID = vec(readdlm("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/ALGP2_Cycle_1_7_Geno_Complete_Impute_JWAS.bin.ids.txt",String))

3205-element Array{String,1}:
 "0001"
 "0002"
 "0004"
 "0005"
 "0006"
 "0007"
 "0009"
 "0010"
 "0011"
 "0012"
 "0013"
 "0014"
 "0015"
 ⋮     
 "3283"
 "3284"
 "3285"
 "3286"
 "3287"
 "3288"
 "3289"
 "3290"
 "3291"
 "3292"
 "3293"
 "3294"

In [14]:
model_equations =
"RN_NurADG1 = Batch+ Died+ EntryAge+ wt + NurPenBatch+SowID
RN_NurADG2 = Batch+ Died+ EntryAge+ wt+ NurPenBatch+SowID
RN_NurADG3 =Batch+ Died+ EntryAge+ wt+ NurPenBatch+SowID
"

R =[0.979209E-02 0 0
0 0.109188E-01 0
0 0 0.106102E-01]
model = build_model(model_equations,R);

set_covariate(model,"EntryAge")
set_covariate(model,"wt")

G1 =[0.979209E-02 0 0
0 0.109188E-01 0
0 0 0.106102E-01]
set_random(model,"NurPenBatch", G1)

G2 =[0.979209E-02 0 0
0 0.109188E-01 0
0 0 0.106102E-01]
set_random(model,"SowID", G2)


In [15]:
G =[0.004 0.002 0.001
0.002 0.005 0.002
0.001 0.002 0.006]
@time add_genotypes(model,M,G,header=true, rowID=rowID, G_is_marker_variance=false)

The marker IDs are set to 1,2,...,#markers
#markers: 435172; #individuals: 3205
 12.561593 seconds (870.71 k allocations: 15.651 GiB, 0.79% gc time)


7.0f0

In [16]:
@time out=runMCMC(model,phenotypes,methods="BayesC",estimatePi = false,
    single_step_analysis = false, pedigree=false, outputEBV = true,
    output_heritability = true, burnin=5000, chain_length = 50000);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Phenotypes for all traits included in the model for individual 1746 in the row 1521 are missing. This record is deleted.
Phenotypes for all traits included in the model for individual 1788 in the row 1559 are missing. This record is deleted.
Phenotypes for all traits included in the model for individual 2327 in the row 1985 are missing. This record is deleted.
The number of observations with both genotypes and phenotypes used in the analysis is 2840.

Pi (Π) is not provided.
Pi (Π) is generated assuming all markers have effects on all traits.

The prior for marker effects covariance matrix is calculated from genetic covariance matrix and Π.
The mean of the prior for the marker effects covariance matrix is:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0



A Linear Mixed Model was build using model equations:

RN_NurADG1 = Batch+ Died+ EntryAge+ wt + NurPenBatch+SowID
RN_NurADG2 = Batc

PosDefException: PosDefException: matrix is not Hermitian; Cholesky factorization failed.